# 0. Dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import plotly.graph_objects as go 
import plotly.express as px #for colors

#prices
from datetime import datetime
import logging
from entsoe import EntsoePandasClient
import pandas as pd

# 1. Data

## 1.1 Profiles

In [ ]:
#utility functions
def sum_numpy(array):
    return round(np.sum(array),2)

def watt_to_kwh(array, timestep_in_hours):
    return array*timestep_in_hours/1000

def make_array_from_df (df, column):
    return df[column].to_numpy()

In [ ]:
#Profile 1:
area_panel_1 = 2 #WP
battery_capacity_1 = 5  # kWh
max_charge_rate_1 = 2  # kW, take negative for discharge

#Profile 2:
area_panel_2 = 5 
battery_capacity_2 = 10  
max_charge_rate_2 = 5  

#Profile 3:
area_panel_3 = 15 #Danny 26..
battery_capacity_3 = 15 #(Danny 16), maybe a bit too much considering below
max_charge_rate_3 = 10

#Change later these 3 profiles and explain how was chosen:
# https://callmepower.be/en/energy/guides/understanding-your-bills
# https://www.solarreviews.com/solar-panel-cost/wisconsin/belgium#best-rated-companies

#robbe profile
area_panel_robbe = 2.2 #6*370
battery_capacity_robbe = 10.2
max_charge_rate_robbe = 3.5

## 1.2 Prices

Format dictionary - not runned for computational efficiency

In [ ]:
class DayAheadPrices:
    def __init__(self, api_key):
        self.client = EntsoePandasClient(api_key=api_key)

    def fetch_and_print_day_ahead_prices(self, start_date, end_date, country_code='BE'):
        """Fetches and prints the day ahead prices for a specified country within a date range."""
        # Convert start_date and end_date to pandas Timestamp with timezone
        timezone = 'Europe/Brussels'
        date_start = pd.Timestamp(start_date, tz=timezone)
        date_end = pd.Timestamp(end_date, tz=timezone)

        logging.info(f'Fetching day ahead prices from ENTSO-E for {country_code}: {date_start} to {date_end}')
        
        # Fetch day ahead prices
        try:
            day_ahead_prices_df = self.client.query_day_ahead_prices(country_code=country_code, start=date_start, end=date_end)
            # Convert the DataFrame to a dictionary with the date as the key and a list of prices as the value
            prices_by_day = {}
            for date, group in day_ahead_prices_df.groupby(day_ahead_prices_df.index.date):
                prices_by_day[str(date)] = group.values.tolist()
                
            print("Day Ahead Prices (€/MWh):")
            print(prices_by_day)
            return prices_by_day
        except Exception as e:
            print(f"An error occurred while fetching data: {e}")

# if __name__ == "__main__":
#     logging.basicConfig(level=logging.INFO)
#     api_key = 'eaccd9d9-2a8e-42b0-9257-2aa47096934e' # Replace with your actual API key
#     day_ahead_prices = DayAheadPrices(api_key)

#     # Example usage: Fetch prices from 2024-03-01 to 2024-03-02 for Belgium
#     start_date = '2022-01-01'
#     end_date = '2023-01-01'
#     prices_fluvius = day_ahead_prices.fetch_and_print_day_ahead_prices(start_date, end_date)

#     start_date = '2023-02-01'
#     end_date = '2023-06-30'
#     robbe_prices = day_ahead_prices.fetch_and_print_day_ahead_prices(start_date, end_date)


Preprocessing is already done... One year full data (if day length not 24 adapted). Also divided by 1000 to get eur/kWh !! 

In [ ]:
# np.savetxt('prices_fluvius.csv', prices_fluvius, delimiter=',') #do not save twice now
# np.savetxt('prices_robbe.csv', robbe_prices, delimiter=',')

## 1.3 robbe

In [ ]:
robbe = pd.read_csv('data/robbe.csv')
robbe_prices = np.loadtxt('data/prices_robbe.csv', delimiter=',')
print(robbe_prices) #142 days

In [ ]:
timestep = 5/60

In [ ]:
robbe #unit here in Watt
#Hoe zou ik deze data moeten gaan gebruiken gegeven de grotere granulariteit??

In [ ]:
consumption_robbe = make_array_from_df(robbe, 'consumption')
production_robbe = make_array_from_df(robbe, 'production')
#replace NaN with 0
consumption_robbe = np.nan_to_num(consumption_robbe)
production_robbe = np.nan_to_num(production_robbe)

In [ ]:
consumption_robbe = watt_to_kwh(consumption_robbe, timestep)
production_robbe = watt_to_kwh(production_robbe, timestep)

In [ ]:
#nan I do not know why?
sum_cons_robbe = sum_numpy(consumption_robbe)
sum_prod_robbe = sum_numpy(production_robbe)
print(f"Total consumption: {sum_cons_robbe} kWh")
print(f"Total production: {sum_prod_robbe} kWh")

In [ ]:
len(consumption_robbe) #same
len(production_robbe)

In [ ]:
max(consumption_robbe) #per 5 min so times 3, OK in line

In [ ]:
#print first and last date and check if there are any missing dates
print(robbe['time'].min())
print(robbe['time'].max())

#5 months
#missing midnight 1 March, ful day 31 March, full day 31 MAy and midnight 1 June

In [ ]:
#plot consumption and production over time using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=robbe['time'], y=consumption_robbe, mode='lines', name='Consumption'))
fig.add_trace(go.Scatter(x=robbe['time'], y=production_robbe, mode='lines', name='Production'))
fig.update_layout(title='Consumption and Production over time', xaxis_title='Date', yaxis_title='Energy (Watt)')

In [ ]:
robbe_prices #[Eur/MWh]

In [ ]:
#put consumption_robbe and production_robbe in a dataframe and add the time column
# robbe_df = pd.DataFrame({'time': robbe['time'], 'consumption': consumption_robbe, 'production': production_robbe})

In [ ]:
#add missing data
#missing midnight 1 March, ful day 31 March, full day 31 MAy and midnight 1 June or LEAVE IT OUT

In [ ]:
# in dataframe maybe easier... with prices ect

## 1.4 Fluvius

### 1.4.1 Consumption

In [ ]:
#explain how Fluvius data has been used, correlation, clustering?,graphs, etc
consumption = pd.read_csv('data/consumption.csv')
#in kWh by default see legend doc

In [ ]:
consumption #do only for one profile, temporary, per quarter so merge it first...

In [ ]:
#for id == 173 make a numpy array with consumption column
cons_1 = consumption[consumption['id'] == 272] #low
cons_1 = cons_1['consumption'].to_numpy()
cons_2 = consumption[consumption['id'] == 173] #median
cons_2 = cons_2['consumption'].to_numpy()
cons_3 = consumption[consumption['id'] == 550] #high
cons_3 = cons_3['consumption'].to_numpy()

In [ ]:
len(cons_3) #quarterly data, all same length

In [ ]:
total_quarters_in_one_year = 365 * 24 * 4
total_quarters_in_one_year

In [ ]:
#print total consumption per cons
sum_numpy(cons_1)
sum_numpy(cons_2)
sum_numpy(cons_3)
print(f'sum cons 1: {sum_numpy(cons_1)}, sum cons 2: {sum_numpy(cons_2)}, sum cons 3: {sum_numpy(cons_3)} kWh/year')

In [ ]:
max(cons_3) #unit is here...kWh

In [ ]:
#plot conss using plotly
#could groupy by 24 values... like in read_data notebook
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(cons_1)), y=cons_1, mode='lines', name='Low - cons 1')) # holidays, maybe not representative, lowest from 1300 conss...
fig.add_trace(go.Scatter(x=np.arange(len(cons_2)), y=cons_2, mode='lines', name='Median - cons 2'))
fig.add_trace(go.Scatter(x=np.arange(len(cons_3)), y=cons_3, mode='lines', name='High - cons 3'))
fig.update_layout(title='Consumption conss', xaxis_title='Quarter', yaxis_title='Consumption (kWh)')
fig.show()


### 1.4.2 Production

In [ ]:
efficiency_loss = 0.5 #tocheck... impact as ell on profile, probably a bit lower
#explain where weather data has been generated

In [ ]:
#load weather data
weather = pd.read_csv('data/weather.csv')

#make columns now for each profile, create first column and then for loop 1 to 4...

In [ ]:
weather

In [ ]:
def calc_prod(df, efficiency_loss): #no need area panel
    #use temp and irradiance to calculate the production in df
    df['prod_one_kw_panel'] = (1 - 0.005 * (df['temperature'] - 25)) * df['solar_irradiation'] * efficiency_loss #make it profile name!!
    print('Done')
    return df

weather_with_prod = calc_prod(weather, efficiency_loss)

In [ ]:
weather_with_prod

In [ ]:
#plot prod_one_kw_panel
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather_with_prod.index, y=weather_with_prod['prod_one_kw_panel']/1000, mode='lines', name='Production')) #here divided by 1000 and hour timestep so fine
fig.update_layout(title='Production of only ONE kW panel', xaxis_title='Time', yaxis_title='Production (kWh)')
fig.show() #need to zoom in

In [ ]:
#print sum of prod_one_kw_panel
print('Total production of 1 kW panel:', weather_with_prod['prod_one_kw_panel'].sum()/1000, 'kWh')
print('Total irrandiance in BE:', weather_with_prod['solar_irradiation'].sum()/1000, 'kWh/m²/year') #W/m² is here one hour timestep, is an instant (average here) delivery of power, not energy
#The yearly solar irradiance in Belgium typically varies around 1,000 to 1,150 kWh per square meter (kWh/m²/year)

#Average consumption in Belgium is 4-5 persons 7,500 kWh/year (profile 2?)

In [ ]:
#create a column prod_1, prod_2, prod_3 in weather_with_prod by multiplying prod_one_kw_panel with area_panel_1, area_panel_2, area_panel_3
weather_with_prod['prod_1'] = weather_with_prod['prod_one_kw_panel'] * area_panel_1
weather_with_prod['prod_2'] = weather_with_prod['prod_one_kw_panel'] * area_panel_2
weather_with_prod['prod_3'] = weather_with_prod['prod_one_kw_panel'] * area_panel_3

In [ ]:
weather_with_prod

In [ ]:
#take array from it and do 4 replication same length normally, divide uniformly and by 1000 (to get KWh per quarter)
#same as make_Wp_to_kWh function
prod_1 = make_array_from_df(weather_with_prod, 'prod_1')/1000
prod_2 = make_array_from_df(weather_with_prod, 'prod_2')/1000
prod_3 = make_array_from_df(weather_with_prod, 'prod_3')/1000

In [ ]:
#plot prod_1, prod_2, prod_3
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather_with_prod.index, y=prod_1, mode='lines', name='Production 1')) #here divided by 1000 and hour timestep so fine
fig.add_trace(go.Scatter(x=weather_with_prod.index, y=prod_2, mode='lines', name='Production 2')) #here divided by 1000 and hour timestep so fine
fig.add_trace(go.Scatter(x=weather_with_prod.index, y=prod_3, mode='lines', name='Production 3')) #here divided by 1000 and hour timestep so fine
fig.update_layout(title='Production of panels', xaxis_title='Time', yaxis_title='Production (kWh) dunring quarter')
fig.show() #need to zoom in

In [ ]:
len(prod_1) 

In [ ]:
#print here below the sum of the production for each profile
print('Total production for profile 1: ', prod_1.sum())
print('Total production for profile 2: ', prod_2.sum())
print('Total production for profile 3: ', prod_3.sum())

### 1.4.3 Prices

In [ ]:
#read from file
prices_fluvius = np.loadtxt('prices_fluvius.csv', delimiter=',')
print(prices_fluvius) #length is 8760

# 2. Parameters

In [ ]:
# #Varying - later

# #Setup
# battery_capacity = [10, 20, 30, 40, 50] # battery_capacity = np.array(battery_capacity) #kWh
# battery_charge_rate= [2.5, 5, 10, 15, 20, 25] #same discharge

# # Forecast and mismatching
# forecast_certainty = [0.5, 0.7, 0.9, 1]
# misatching_prob = [0, 0.2, 0.4, 0.5] #during day?

# #Num of guesses
# num_guesses = [10, 20, 30] #epending on hoirzon

# #Timesteps
# do I need to assume a certain granularity?
# frequency_granualirty = [24, 12, 8, 6, 4, 2, 1, 1/2, 1/4, 1/8, 1/12]
# horizon_in_timesteps =  [0, 1, 3, 5] #also remaining time or maybe procentual

# #Prices ell back ratio
# sell_back_ratio = [0.2, 0.5, 0.8, 1]

# #Seasonality is embedded, explain empirical case
# #try to minimise other params, methods, options explained was experimented before no significant difference, work procentually
#initial state_battery, no impact I guess

# #Goal: find patterns where mismatching has an impact depending on parameters, quantify the problem and say how condense it

# # do a foor loop for testing everything and keeping the record.... nmber of combinations ..


In [ ]:
max_capacity = 10  # kWh
max_charge_rate = 5  # kWh per hour (kW)
max_discharge_rate = -5 #negative!

#only this to change

timesteps_for_mismatching = 2
probability_mismatching = 0.1 #4 scenarios, no 0.2, 0.5, 1

efficiency_battery = 1
efficiency_inverter = 1 
grid_price_ratio = 0.5
fraction_std_dev = 8

#mismatching when above average assumption
mismatching_occurence = 1 #0 both, 1 pv, 2 consumption

# Stored elec in battery and directly consumed(c) PV go through inverter
# PV production (p) to battery no need to go through inverter

# Initial state
soc_start = 0  # not kWh value, but percentage
horizon = 10 # number of timesteps
forecast_certainity = 1 #percentage

# for MPC
options = {'maxiter': 10, 'ftol': 0.2} #consequtive
method = 'SLSQP'
num_guesses = 10

In [ ]:
N = 2 #duration in hours of a timestep!!!!

# 3. Data aggregation

First start with data from robbe, so take 15 min as anchor

2 functions: todo make it more generic

N can be 1/12, 1/4, 1/2, 1, 2, 6, 12, 24

5, 15, 30 min 1 uur, 2 uur , 6 uur, 12 uur, 24 uur

- Length prices: 24 (capped), 48, 96, 192, 288
- Length Cons/prod: 1, 2, 3, 6, 12, 24, 48, 96, 192, 288

In [ ]:
#make production quarterly

def prod(array):
    replication_factor = 4
    array = np.repeat(array, replication_factor) / 4
    return array

In [ ]:
def prices_dismantle(prices, duration_timestep_in_hours):
       
    if duration_timestep_in_hours >= 1: #always know the prices
        return prices
    else: #duration_timestep_in_hours < 1
        repeat_factor = 1/duration_timestep_in_hours
        prices = np.repeat(prices, repeat_factor)
        
        return prices

In [ ]:
def resize_prod_cons_fluvius(arr, duration_timestep_in_hours): #in functie lengte van arr for robbe

    initial_length = len(arr) #35040 or 105120

    if duration_timestep_in_hours == 1/12:
        repeat_factor = 3
        arr = np.repeat(arr, repeat_factor)
        arr = arr / repeat_factor
        return arr
    
    elif duration_timestep_in_hours == 1/4:
        return arr
    
    elif duration_timestep_in_hours == 1/2:
        #every two elements take the sum (not mean)
        new_arr = np.sum(arr.reshape(-1, 2), axis=1)
        return new_arr
    
    elif duration_timestep_in_hours == 1:
        new_arr = np.sum(arr.reshape(-1, 4), axis=1)
        return new_arr
    
    else: #major assumptions hourly prices are known!!
        #N 2,6, 12 or 24
        #aggregate and distribute
        repeat_factor = duration_timestep_in_hours * 4
        new_arr = np.sum(arr.reshape(-1, repeat_factor), axis=1)
        new_arr = np.repeat(new_arr, duration_timestep_in_hours)
        new_arr = new_arr / duration_timestep_in_hours
        if len (new_arr) != 8760:
            print('Error in resizing')
        return new_arr
    

#todo for robbe

### From here for data computations...

In [ ]:
electricity_prices = prices_fluvius  # Euro per kWh
pv_production = prod_2 # PV production (kWh)
consumption = cons_2  # Household consumption (kWh)

#later in parameter do something with profile

N = 2
timesteps_for_mismatching = 3
horizon = 6

In [ ]:
#change if robbe data!!
#do not run it twice...

if len(pv_production) == 24*365:
    pv_production = prod(pv_production) #for prod_1, prod_2, prod_3

electricity_prices = prices_dismantle(electricity_prices, N)
pv_production = resize_prod_cons_fluvius(pv_production, N)
consumption = resize_prod_cons_fluvius(consumption, N)
print(electricity_prices)
print(pv_production)
print(consumption)
#check all nmpy array same length

if len(electricity_prices) != len(pv_production) or len(electricity_prices) != len(consumption):
    
    print(len(electricity_prices))
    print(len(pv_production))
    print(len(consumption))
    raise ValueError("All numpy arrays must have the same length")



In [ ]:
#keep only the first 1000 values of each
electricity_prices = electricity_prices[5000:5500]
pv_production = pv_production[5000:5500]
consumption = consumption[5000:5500]

#make full sample data for visualisation copy
electricity_prices_full = electricity_prices
pv_production_full = pv_production
consumption_full = consumption

In [ ]:
print(len(electricity_prices))
print(len(pv_production))
print(len(consumption))

# Data simulation - temporary - outcommented

## Data creation

In [ ]:
# #imagine for the moment fake data quarterly (4*24) everything!!!
# G = 4*24 #granularity of the data
# np.random.seed(42)
# #prices always 24
# electricity_prices = np.random.uniform(0.1, 0.6, 24)  # Euro per kWh
# electricity_prices = np.round(electricity_prices, decimals=1) 
# pv_production = np.random.randint(0, 5, G) # PV production (kWh)
# consumption = np.random.randint(0, 20, G)  # Household consumption (kWh)

In [ ]:
# # N = 24 #duration of a timestep in hours ! (changed) # for simulation

# max_capacity = 10  # kWh
# max_charge_rate = 5  # kWh per hour (kW)
# max_discharge_rate = -5 #negative!

# #only this to change

# timesteps_for_mismatching = 2
# probability_mismatching = 0.1 #1- zou moeten zijn, also difference between production and consumption

# efficiency_battery = 1
# efficiency_inverter = 1 
# grid_price_ratio = 0.5
# fraction_std_dev = 8

# # Stored elec in battery and directly consumed(c) PV go through inverter
# # PV production (p) to battery no need to go through inverter

# # Initial state
# soc_start = 0  # not kWh value, but percentage
# horizon = 10 # number of timesteps
# forecast_certainity = 1 #percentage

# # for MPC
# options = {'maxiter': 1000, 'ftol': 1e-6}
# method = 'SLSQP'
# num_guesses = 25

## Data aggregation

In [ ]:
def prices_dismantle(prices, t):
       
    if t >= 1: #always know the prices
        return prices
    else: #t < 1
        new_length = 24/t
        if new_length % 24 == 0:
            repeat_factor = new_length // 24 #already ensured multiple of 24 in cons and prod, should be an integer
            new_price = np.repeat(prices, repeat_factor)
            return new_price
        else:
            raise ValueError("timestep is not a multiple of 24")

In [ ]:
def resize_prod_cons(arr, t):
    new_length = int(24/t)
    # print(f'from 96 to length of: {new_length}')
    list_allowed_lenghts = [1, 2, 3, 4, 6, 12, 24, 48, 96, 192, 288]

    if new_length not in list_allowed_lenghts:
        raise ValueError("New length must be an integer multiple or divisible by the original length.")
    
    else:
        if new_length == 96:
            return arr
        elif new_length > 96:
            repeat_factor = new_length // 96
            new_arr = np.repeat(arr, repeat_factor)
            #here also would need to divide by repeat factor
            return new_arr
        elif new_length < 96:
            group_size = 96 // new_length
            new_arr = np.sum(arr.reshape(-1, group_size), axis=1) #here should be sum not mean

            if new_length < 24:
                redistribute_factor = 24 // new_length
                new_arr = np.repeat(new_arr, redistribute_factor) #step 1: repeat
                new_arr = new_arr / redistribute_factor #step 2: divide

            return new_arr

In [ ]:
# # N can be 24, 12, 8, 6, 4, 2, 1, 1/2, 1/4, 1/8, 1/12
# # to do convert in minutes
# N= 2

# electricity_prices = prices_dismantle(electricity_prices, N)
# pv_production = resize_prod_cons(pv_production, N)
# consumption = resize_prod_cons(consumption, N)
# print(electricity_prices)
# print(pv_production)
# print(consumption)
# #check all nmpy array same length
# if len(electricity_prices) != len(pv_production) or len(electricity_prices) != len(consumption):
#     raise ValueError("All numpy arrays must have the same length")

# #make full sample data for visualisation copy
# electricity_prices_full = electricity_prices
# pv_production_full = pv_production
# consumption_full = consumption

# print(f'length of electricity_prices: {len(electricity_prices)}')

# 4. MPC for battery scheduling

3 main constraints:
- demand = supply at t taking into account battery and PV
- cannot output charging command that exceeds soc boundaries in next timestep 
- properties respected: max charging rate - corrected with efficiency

Note: consider always production, price, consumption for the COMING timestep. SOC will be one element extra bcs initial

For later (inverter and dynamic grid prices, aggregation of timesteps to see impact)

## 4.1 Utility functions

In [ ]:
# Utility functions defined outside the class
def total(price, volume):
    return price * volume

def generate_profiles(time_segments, probability): #add pprobab as param
        np.random.seed(42)

        if np.random.rand() > probability:  # 90% of the time, add this as a parameter later
                profile = np.full(time_segments, 1 / time_segments)  # Uniformly distributed
        else:
            random_values = np.random.uniform(0, 1, time_segments)
            random_values /= random_values.sum()  # Ensure sum is 1
            profile = np.round(random_values, decimals=1)  # Rounded to one decimal

        return profile

## 4.2 Class: among other cost function and scenarios

N becomes T, no self.T anymore

In [ ]:
class OptimizationProblem:

    #note name variables in __init__ are not the same as the ones in the class
    def __init__(self, soc_initial, electricity_prices, pv_production, consumption, N, horizon, max_capacity, max_charge_rate, max_discharge_rate, efficiency_battery, efficiency_inverter, grid_price_ratio, fraction_std_dev, time_segments, method, options, num_guesses, probs_mismatch, accuracy_forecast, mismatching_occurence):
        self.soc_initial = soc_initial
        self.H = horizon
        self.N = N #length of one timestep in hours


        #I think below for simulation data but not needed
        # if N > 1:
        #     self.N = 24
        # else:
        #     self.N = int(24/N) #number of timesteps
        #--------
        #N is the input, used for length of timestep in hours (T) and number of timesteps also (N)!!
        #H is horizon

        self.electricity_prices = electricity_prices
        self.pv_production = pv_production
        self.consumption = consumption

        self.max_capacity = max_capacity
        self.max_charge_rate = max_charge_rate
        self.max_discharge_rate = max_discharge_rate
        self.efficiency_inverter = efficiency_inverter
        self.efficiency_battery = efficiency_battery

        self.time_segments = time_segments
        self.probs_mismatch = probs_mismatch
        self.grid_price_ratio = grid_price_ratio
        self.fraction_std_dev = fraction_std_dev
        self.accuracy_forecast = accuracy_forecast
        self.mismatching_occurence = mismatching_occurence

        self.method = method
        self.options = options
        self.num_guesses = num_guesses

        self.true_charging_actions = None  # To store the "true" charging actions
        self.soc_list = None  # To store the state of charge over time
        self.grid_list = None  # To store the net grid interaction over time

        #for looping over best guesses
        self.best_actions = None
        self.best_soc = None
        self.best_grid = None
        self.best_fake_actions = None

        self.evaluations = [] #for guesses convergence

    def get_bounds(self):
        # Generate bounds based on max_charge_rate and max_discharge_rate
        return [(self.max_discharge_rate, self.max_charge_rate) for _ in range(self.H)]
    
    #changed to horizon
    

    def initial_guesses(self):
        current_soc = self.soc_initial
        if num_guesses == 1:
            # One guess: neutral middle-ground, rounded to two decimals.
            return [np.round(np.zeros(self.H), 2)]
        else:
            guesses = []
            for _ in range(num_guesses):
                guess = np.zeros(self.H)
                for t in range(self.H):
                    # Randomly decide between discrete choices and a continuous range selection.
                    if np.random.rand() < 0.25:
                        # 20% chance to select any value in the range.
                        action = np.random.uniform(self.max_discharge_rate, self.max_charge_rate)
                    else:
                        # 80% chance to favor extremes and zero.
                        if current_soc == 0:
                            # Can only charge_rate.
                            action = np.random.choice([0, self.max_charge_rate], p=[0.2, 0.8])
                        elif current_soc == self.max_capacity:
                            # Can only discharge_rate.
                            action = np.random.choice([0, self.max_discharge_rate], p=[0.2, 0.8])
                        else:
                            # Can charge_rate or discharge_rate; favor extremes and zero.
                            action = np.random.choice([0, self.max_charge_rate, self.max_discharge_rate], p=[0.4, 0.3, 0.3])

                    # Round the action to two decimals.
                    action = np.round(action, 2)
                    
                    # Apply action to the current state of charge_rate, ensuring it remains within bounds.
                    current_soc = np.clip(current_soc + action, 0, self.max_capacity)
                    guess[t] = action
                    
                    # Optionally, enforce more realistic sequences (e.g., discharge_rate after charging).
                    if t > 0 and guess[t-1] > 0 and action > 0:
                        # If previously charging, introduce a higher chance to switch to discharging or neutral.
                        if np.random.rand() < 0.5:
                            guess[t] = np.random.choice([0, self.max_discharge_rate], p=[0.5, 0.5])
                            guess[t] = np.round(guess[t], 2)
                    
                    current_soc = np.clip(current_soc + guess[t] - action, 0, self.max_capacity)
                
                guesses.append(guess)
            
            return guesses

    
    def heuristic(self): #because they change over time
        sum_pv = np.sum(self.pv_production)
        sum_cons = np.sum(self.consumption)
        battery = self.max_capacity * (self.soc_initial)
        difference = sum_cons - sum_pv - battery

        print(f'sum_pv: {sum_pv}, sum_cons: {sum_cons}, battery: {battery}, difference: {difference}')

        max_price = np.max(self.electricity_prices)
        min_price = np.min(self.electricity_prices)
        avg_price = np.mean(self.electricity_prices)
        max_cost = total(max_price, difference * self.N)
        min_cost = total(min_price, difference * self.N)
        avg_cost = total(avg_price, difference * self.N)
        return max_cost, min_cost, avg_cost

        '''Change to one heursitic: difference each timestep : net grid times the corresponding price - so basically no battery benchmark''' #mybe only one heuristic

    #core, I only need SOc, actions and net_grid to come out of this, rest can be computed afterwards
    def cost_function(self, proposed_charging_actions):
        soc = self.soc_initial
        charging_cost = 0
        soc_list = [soc]  # Initialize SOC list with initial SOC
        charging_action_list = []
        net_grid = []

        #constraint 2
        for i in range(self.H): #at each timestep

            #changed to horizon

            #a) action
            # -------------------------------------
            if proposed_charging_actions[i] > 0:  # Charging
                charging_action = min(proposed_charging_actions[i] * self.N, (1 - soc) * self.max_capacity) 
                # print(f'proposed_charging_actions {proposed_charging_actions[i]} and self.N {self.N}')
                # print(f'charging action: {charging_action}')
                #not necessary to adapt for efficiency battery, it is LOSS
            elif proposed_charging_actions[i] < 0:  # Discharging
                charging_action = max(proposed_charging_actions[i] * self.N, -soc * self.max_capacity) #maximum of two negative values. Quesrion: can I send signal to battery to discharge more?
                # print(f'discharging action: {charging_action}')
            else:  # No charging action
                # print('no action')
                charging_action = 0
            
            #b) state of charge
            # -------------------------------------
            if charging_action > 0:
                real_action = charging_action * self.efficiency_battery 
                soc += real_action / self.max_capacity
            elif charging_action < 0:  # When you ask a battery to discharge a certain power, taking into account its efficiency, it needs to discharge more than requested to ensure the desired output is met due to efficiency losses
                real_action = charging_action / self.efficiency_battery
                soc += real_action / self.max_capacity
            else:
                real_action = 0
            
            # DID not take into account inverter yet
            # soc = max(0, min(soc, self.max_capacity))  # not strictly necessary
            
            #c) net grid
            # -------------------------------------
            # constraint 1
            grid = self.consumption[i] - self.pv_production[i] + charging_action

            #chargig_cost is aggregate, later use ut to calculate buyinfg and selling
            if grid > 0:
                charging_cost += grid * self.electricity_prices[i] * self.N #because price is per hour eur/kWh
            elif grid < 0:
                charging_cost += grid * self.grid_price_ratio * self.electricity_prices[i] * self.N
            else:
                pass

            # append to lists
            soc_list.append(round(soc,2))
            charging_action_list.append(round(real_action, 2))
            net_grid.append(round(grid,2))

        # Update the instance attributes with the final results
        self.true_charging_actions = charging_action_list
        self.soc_list = soc_list
        self.grid_list = net_grid

        self.evaluations.append((charging_cost)) #only keep the first see below

        return charging_cost
        # Return the total charging cost as the optimization objective (only one variable allowed)
    

    def apply_mismatch_and_update_soc(self, prod_amount, cons_amount, proposed_action, soc_init): #not needed to be in class I think, but better for params
        #initialize
        #no need efficiency already done in real_action in cost function!!!
        # np.random.seed(42) in generate profile
        print('hello')
        soc = soc_init #not same as self.soc_initial
        net_grid_segment = 0
        from_grid = 0
        to_grid = 0
        total_grid = []
        actions_segment_list = []

        # Generate mismatch profiles
        if self.mismatching_occurence == 0: 
            prod_mismatch_profile = generate_profiles(self.time_segments, self.probs_mismatch)
            cons_mismatch_profile = generate_profiles(self.time_segments, self.probs_mismatch)
        elif self.mismatching_occurence == 1:
            prod_mismatch_profile = generate_profiles(self.time_segments, self.probs_mismatch)
            cons_mismatch_profile = np.full(self.time_segments, 0)
        else: #self.mismatching_occurence == 2
            prod_mismatch_profile = np.full(self.time_segments, 0)
            cons_mismatch_profile = generate_profiles(self.time_segments, self.probs_mismatch)

        # print(f'prod_mismatch_profile: {prod_mismatch_profile}')
        # print(f'cons_mismatch_profile: {cons_mismatch_profile}')

        # Iterate through each time segment
        for i in range(self.time_segments):
            # print(f'this is the {i}th iteration')
            segment_prod = prod_amount * prod_mismatch_profile[i]
            segment_cons = cons_amount * cons_mismatch_profile[i]
        
            if proposed_action > 0:  # Charging
                action_for_segment = min(proposed_action * prod_mismatch_profile[i], max_charge_rate * self.N/self.time_segments) #here divided by 2 because hour, be careful later to not interchange charge rate and capcity kw and kwh 
                soc += action_for_segment / self.max_capacity

            else:  # Discharging (or no action)
                action_for_segment = max(proposed_action * cons_mismatch_profile[i], self.max_discharge_rate * self.N/self.time_segments) #same comments, max of two negative numbers
                soc += action_for_segment / self.max_capacity
            
            # Update net grid exchange
            net_grid_segment = segment_cons - segment_prod + action_for_segment

            if net_grid_segment > 0:
                from_grid += net_grid_segment
            else:
                to_grid += net_grid_segment

            actions_segment_list.append(action_for_segment)
    
            # soc = max(0, min(soc, 1)) * 100 #not needed normally

            #prints
            # print(f'new soc at end segment {round(soc * 100, 2)} % , action in segment: {round(action_for_segment,2)}, segment cons: {round(segment_cons,2)}, segment prod: {round(segment_prod,2)}, net grid: {round(net_grid_segment,2)}')
        
        total_grid = [from_grid, to_grid]

        return soc, total_grid, actions_segment_list
    

    #-------------------------- # not a lot different from cost function

    def basic_ruler(self):
        soc = self.soc_initial
        charging_cost = 0
        soc_list = [soc]  # Initialize SOC list with initial SOC
        charging_action_list = []
        net_grid = []
        mean_price = np.mean(self.electricity_prices)

        #constraint 2
        for i in range(len(self.consumption)): #at each timestep, nothing to do with horizon bcs kijkt niet op voorhand, mismatching is per action donc blc

            #a) action lined to price
            # -------------------------------------
            if self.electricity_prices[i] < mean_price:  # Charging
                charging_action = min(self.max_charge_rate * self.N, (1 - soc) * self.max_capacity) 
            
            elif self.electricity_prices[i] >= mean_price:  # Discharging
                charging_action = max(self.max_discharge_rate * self.N, -soc * self.max_capacity) 
            else:  # No charging action
                charging_action = 0
                print('not possible')
            
            #b) state of charge
            # -------------------------------------
            if charging_action > 0:
                real_action = charging_action * self.efficiency_battery 
                soc += real_action / self.max_capacity
            elif charging_action < 0:  # When you ask a battery to discharge a certain power, taking into account its efficiency, it needs to discharge more than requested to ensure the desired output is met due to efficiency losses
                real_action = charging_action / self.efficiency_battery
                soc += real_action / self.max_capacity
            else:
                real_action = 0
            
            # DID not take into account inverter yet
            # soc = max(0, min(soc, self.max_capacity))  # not strictly necessary
            
            #c) net grid
            # -------------------------------------
            # constraint 1
            grid = self.consumption[i] - self.pv_production[i] + charging_action

            #chargig_cost is aggregate, later use ut to calculate buying and selling
            if grid > 0:
                charging_cost += grid * self.electricity_prices[i]
            elif grid < 0:
                charging_cost += grid * self.grid_price_ratio * self.electricity_prices[i]
            else:
                pass

            # pend to lists
            soc_list.append(round(soc,2))
            charging_action_list.append(round(real_action, 2))
            net_grid.append(round(grid,2))

        # Update the instance attributes with the final results
        self.true_charging_actions = charging_action_list
        self.soc_list = soc_list
        self.grid_list = net_grid

        self.evaluations.append((charging_cost)) #only keep the first see below

        return charging_cost
        # Return the total charging cost as the optimization objective (only one variable allowed) - Here the return is not really necessary!

## 4.3 Optimisation def

In [ ]:
def optimize_battery_operation(problem: OptimizationProblem, soc):

    initial_guesses = problem.initial_guesses()
    print(initial_guesses)

    bounds = problem.get_bounds() #will need to be changed later
    best_result = None

    list_nfev = [] #maybe do a list for each guess

    for guess in initial_guesses:
        problem.soc_initial = soc
        result = minimize(
            problem.cost_function, 
            guess,  
            method= problem.method, 
            bounds=bounds,
            options= problem.options
        )

        list_nfev.append(result.nfev)

        if best_result is None or result.fun < best_result.fun:
            best_result = result
            problem.best_fake_actions = result.x
            problem.best_actions = problem.true_charging_actions
            problem.best_soc = problem.soc_list
            problem.best_grid = problem.grid_list
    
    print(f"Best result: {round(best_result.fun,2)} Euro")
    # print("Best proposed fake optimal actions:", proposed_fake_optimal_actions)
    # print('Number of iterations', result.nfev)

    return list_nfev #maybe need to change this

## 4.4 Cost computation

In [ ]:
def cost_calculator_no_mismatching(net_grid, electricity_prices, ratio_sell_back, timestep_duration):
    # input: net_grid, ratio_sell_back, prices
    total_cost = 0
    buying_cost = 0
    selling_cost = 0

    if len(net_grid) != len(electricity_prices):
        repeat_factor = len(electricity_prices) // len(net_grid)
        electricity_prices = np.mean(electricity_prices.reshape(-1, repeat_factor), axis=1)
        print(f'Length of net_grid and electricity_prices are set the same {electricity_prices}')

    
    for i in range(len(net_grid)):
        if net_grid[i] > 0:
            buying_cost += net_grid[i] * electricity_prices[i] * timestep_duration
        elif net_grid[i] < 0:
            selling_cost += net_grid[i] * ratio_sell_back * electricity_prices[i] * timestep_duration #negative I think
        else:
            pass

    total_cost = buying_cost + selling_cost

    return total_cost, buying_cost, selling_cost

In [ ]:
def cost_calculator_with_mismatching(net_grid, electricity_prices, ratio_sell_back, timestep_duration):
    # input: net_grid, ratio_sell_back, prices
    total_cost = 0
    buying_cost = 0
    selling_cost = 0
    
    if len(net_grid) != len(electricity_prices):
        repeat_factor = len(electricity_prices) // len(net_grid)
        electricity_prices = np.mean(electricity_prices.reshape(-1, repeat_factor), axis=1)
        print(f'Length of net_grid and electricity_prices are set the same {electricity_prices}')
    
    buy, sell = zip(*net_grid)
    # print(f'buy: {buy}, sell: {sell}')
    
    buying_cost = np.sum(buy * electricity_prices * timestep_duration)
    selling_cost = np.sum(sell * electricity_prices * timestep_duration) * ratio_sell_back
    total_cost = buying_cost + selling_cost

    return total_cost, buying_cost, selling_cost

## 4.5 Benchmarks - need to incorporate the mismatching (later if time)

Before problem deployment so still fine in terms of variables, seed in mismatching so fine

### No battery

Basic difference of actual consumption minus production and multiply by price

In [ ]:
#defined before at the end data aggregation section
# electricity_prices_full = electricity_prices
# pv_production_full = pv_production
# consumption_full = consumption

In [ ]:
# electricity_prices #still fine

#problem class init

In [ ]:
if len(consumption_full) != len(pv_production_full) != len(electricity_prices_full):
    raise ValueError("All numpy arrays must have the same length")
net_grid = consumption_full - pv_production_full #if same length
total_no_battery, buy_no_battery, sell_no_battery = cost_calculator_no_mismatching(net_grid, electricity_prices_full, grid_price_ratio, N)
print(f'total: {total_no_battery}, buy: {buy_no_battery}, sell: {sell_no_battery}')

In [ ]:
#for mismatching apply apply_mismatch_and_update_soc
# but proposed action is always zero. In return keep only the grid evolution for grid computation...

### Basic rule based controller

In [ ]:
#mean of electricity_prices
mean_price = np.mean(electricity_prices_full)

In [ ]:
# If prices lower than average? for instance charge...
# If higher than average discharge.. 
#always at max rate
# keep track of cost and soc

In [ ]:
basic_ruler = OptimizationProblem(soc_start, electricity_prices, pv_production, consumption, N, horizon, max_capacity, max_charge_rate, max_discharge_rate, efficiency_battery, efficiency_inverter, grid_price_ratio, fraction_std_dev, timesteps_for_mismatching, method, options, num_guesses, probability_mismatching, forecast_certainity, mismatching_occurence)
#won't use everything every argument

total_basic_ruler = basic_ruler.basic_ruler()
print(f'total_basic_ruler: {total_basic_ruler}')


Optimal controller a bit less... but worse with mismatching so need to implement this! Do both but actually like looping problem

Charge by night and if excess store
Check old project ML6

# 5. Problem deployment

## 5.1 no loop problem - BUT now only computing the firt horizon... so not all timesteps like before...

see visualisation graph 4

this does not mean a lot...

In [ ]:
horizon_no_loop = int(24/(3* N)) #this forces 24 hour lookup so you can decrease by a lot, at least 2... Even 10 is a lot
no_loop_problem = OptimizationProblem(soc_start, electricity_prices, pv_production, consumption, N, horizon_no_loop, max_capacity, max_charge_rate, max_discharge_rate, efficiency_battery, efficiency_inverter, grid_price_ratio, fraction_std_dev, timesteps_for_mismatching, method, options, num_guesses, probability_mismatching, forecast_certainity, mismatching_occurence)
no_loop_list_nfev_guesses = optimize_battery_operation(no_loop_problem, soc_start)

# class to be able to plot this
# print("Not time-corrected actions:", no_loop_problem.best_fake_actions)
# print("Best true charging actions:", no_loop_problem.best_actions)
# print("Best SOC list:", no_loop_problem.best_soc) #one extra value
# print("Best net grid list:", no_loop_problem.best_grid)

# no_loop_max_heuristic, no_loop_min_heuristic, no_loop_avg_heuristic = no_loop_problem.heuristic()
# #difference cannot be zero... otherw
# ise all heuristics zero
# print(f'heuristic max: {no_loop_max_heuristic}, heuristic min: {no_loop_min_heuristic}, heuristic avg: {no_loop_avg_heuristic}')

In [ ]:
# no_loop_list_nfev_guesses

In [ ]:
# options = {'maxiter': 20, 'ftol': 0.1, 'eps': 0.5} #consequtive
# method = 'SLSQP'
# num_guesses = 10 #if smaller horizon less guesses needed...
# #show graph trade-off on an example...

# #also horizon and num of mismatching !!... test only extreme cases
# # horizon_no_loop = int(24/(3* N)) #so horizon is like 1,2,3,4 part of the day you know in advance... but no sure becasue 15 min and 30 min quite lot still

In [ ]:
# sum(no_loop_list_nfev_guesses) #should be 25

Note: charging actions are now updated based on N (duration of timestep)

In [ ]:
cost_no_loop, buy_no_loop, sell_no_loop = cost_calculator_no_mismatching(no_loop_problem.best_grid, electricity_prices_full[:horizon_no_loop], grid_price_ratio, N)

This cost does not mean anything!!!

In [ ]:
print(f'total_no_loop: {cost_no_loop}, buy_no_loop: {buy_no_loop}, sell_no_loop: {sell_no_loop}')

## 5.2 rolling problem

The core idea is to adapt the existing framework to dynamically update and apply the optimal control actions in a sequential manner, reflecting a more realistic operational scenario where decisions are made in real-time based on the latest information.

In [ ]:
#forecast variability, no discrimination between prod and cons for the mooment, prices are known
#utility function

def stochastically_deviate(array, deviation_range_percentage):
    
    # For each element, calculate a deviation percentage within the allowed range
    deviations = np.random.uniform(-deviation_range_percentage, deviation_range_percentage, size=array.shape)
    
    # Apply the calculated deviations to the original array elements
    deviated_array = array + (array * deviations)
    
    return deviated_array

# # Example usage
# horizon = 3
# original_array = np.array([1, 2, 4])
# probability = 0.1  # This means each element in the array can deviate up to 90% of its value

# # Apply the stochastic deviation function
# deviated_array = stochastically_deviate(original_array, probability)
# deviated_array


Can add some prints in class and need to rerun everything

In [ ]:
mismatching_occurence = 0

In [ ]:
# Initialize the rolling_problem instance with the initial parameters, put it here to able to rerun it
rolling_problem = OptimizationProblem(soc_start, electricity_prices, pv_production, consumption, N, horizon, max_capacity, max_charge_rate, max_discharge_rate, efficiency_battery, efficiency_inverter, grid_price_ratio, fraction_std_dev, timesteps_for_mismatching, method, options, num_guesses, probability_mismatching, forecast_certainity, mismatching_occurence)

rolling_max_heuristic, rolling_min_heuristic, rolling_avg_heuristic = rolling_problem.heuristic() #do it here otherwise prices, prod and cons will change

horizon_copy = rolling_problem.H
length_copy = len(rolling_problem.electricity_prices)
print(f'length of electricity_prices: {length_copy}, horizon: {horizon_copy}')

# Prepare for rolling horizon optimization
soc = rolling_problem.soc_initial

optimal_controls_without_mismatching = []
soc_evolution_without_mismatching = [soc]
net_grid_evolution_without_mismatching = []

optimal_controls_with_mismatching = []
soc_evolution_with_mismatching = [soc]
net_grid_evolution_with_mismatching = [] #maybe not needed

keep_only_first = True
number_iterations_per_time_step = []

for t in range(length_copy):
# for t in range(3):

    # Subset data for the remaining part of the day
    #here instead could only recreate data...SIMULATION FOrECASTS variability
    print(f'this is scheduling at begin timestep {t}') 
    value = rolling_problem.N
    # print(value)

    # rolling_problem.electricity_prices = rolling_problem.electricity_prices[-value:] #not t,1,0 because evoluate with time (below)
    # rolling_problem.pv_production = rolling_problem.pv_production[-value:]
    # rolling_problem.consumption = rolling_problem.consumption[-value:]

    rolling_problem.electricity_prices = electricity_prices_full[t:t+horizon_copy] #not t,1,0 because evoluate with time (below)
    rolling_problem.pv_production = pv_production_full[t:t+horizon_copy]
    rolling_problem.consumption = consumption_full[t:t+horizon_copy]

    if len(rolling_problem.electricity_prices) != horizon_copy: 
        #append zeros
        rolling_problem.electricity_prices = np.append(rolling_problem.electricity_prices, np.zeros(horizon_copy - len(rolling_problem.electricity_prices)))
        rolling_problem.pv_production = np.append(rolling_problem.pv_production, np.zeros(horizon_copy - len(rolling_problem.pv_production)))
        rolling_problem.consumption = np.append(rolling_problem.consumption, np.zeros(horizon_copy - len(rolling_problem.consumption)))


    #problem here is N that evoluates with it but instead should do with horizon variable
    if rolling_problem.accuracy_forecast != 1:
        deviation = 1 - rolling_problem.accuracy_forecast
        rolling_problem.pv_production = stochastically_deviate(rolling_problem.pv_production, deviation)
        rolling_problem.consumption = stochastically_deviate(rolling_problem.consumption, deviation)
    
    '''Here add forecasts variability and horizion (class rolling versus stable data)'''
    #---------------here could add forecasts variability

    # print(f'elec prices {rolling_problem.electricity_prices}')
    # print(f'pv production {rolling_problem.pv_production}')
    # print(f'consumption {rolling_problem.consumption}')
   

    # Optimize battery operation for the remaining part of the day, guesses automatically generated
    list_nfev_guesses = optimize_battery_operation(rolling_problem, soc)

    number_iterations_per_time_step.append(list_nfev_guesses) #idependent of mismatching

    #------- 
    if keep_only_first: #only for plotting, initalized on True first (above=
        list_nfev = list_nfev_guesses
        charging_cost_convergence = rolling_problem.evaluations.copy()

        keep_only_first = False
        # first_schedule = rolling_problem.best_actions
        # best_result = best_result.fun
        # print(f'first_schedule: {first_schedule}')
        # print(f'best_result: {best_result}')
    #-------

    """ WITHOUT MISMATCHING"""
    optimal_controls_without_mismatching.append(rolling_problem.best_actions[0])
    soc_evolution_without_mismatching.append(rolling_problem.best_soc[1]) 
    net_grid_evolution_without_mismatching.append(rolling_problem.best_grid[0])
    
    """WITH MISMATCHING"""

    '''Todo different mismatching scenarios ex. not by night'''
    real_soc, real_grid, real_actions = rolling_problem.apply_mismatch_and_update_soc(rolling_problem.pv_production[0], rolling_problem.consumption[0], rolling_problem.best_actions[0], soc)

    #keep track of evolutions, here lists of lists
    optimal_controls_with_mismatching.append(real_actions)
    net_grid_evolution_with_mismatching.append(real_grid) #real_grid[0] is always from grid, real_grid[1] is always to grid
    soc_evolution_with_mismatching.append(real_soc)
    
    #update soc
    soc = real_soc
    #-------------------
    # update remaining time
    # rolling_problem.N = rolling_problem.N - 1 #not needed anymore


# 6. Visualisation

Problem with my model... I cannot charge my battery and send elec back to net at same time? Jawel

I need:
- price, consumption, production. 
- SOC and actions evolution
- Cost (taking and savings) - kan achteraf berekend worden if you have net_grid_consumption... MOST interesting


How to visualise:
- Plotly price, net consumption, consumption, production
- commands and state of battery (two axis)

In [ ]:
#add prints
print('FINAL RESULTS')
print(f'number_iterations_per_time_step: {number_iterations_per_time_step}')
print(f'optimal_controls_without_mismatching: {optimal_controls_without_mismatching}')
print(f'soc_evolution_without_mismatching: {soc_evolution_without_mismatching}')
print(f'net_grid_evolution_without_mismatching: {net_grid_evolution_without_mismatching}')
print(f'optimal_controls_with_mismatching: {optimal_controls_with_mismatching}')
print(f'soc_evolution_with_mismatching: {soc_evolution_with_mismatching}')
print(f'net_grid_evolution_with_mismatching: {net_grid_evolution_with_mismatching}')

## 6.1 Preprocessing: adapt timeseries length to soc series

In [ ]:
#create a function to add a time step (copy of the lat element) if len time series is not equal to len SOC
def equalize_series_length(time_series, soc_series):
    if len(time_series) != len(soc_series):
        # Add an extra time step by appending the last value of the time series
        time_series = np.append(time_series, time_series[-1])
    return time_series

In [ ]:
equ_optimal_controls_without_mismatching = equalize_series_length(optimal_controls_without_mismatching, soc_evolution_without_mismatching)
# optimal_controls_with_mismatching = [sum(inner_list) for inner_list in optimal_controls_with_mismatching] #quid how to visualise...
# optimal_controls_with_mismatching = equalize_series_length(optimal_controls_with_mismatching, soc_evolution_with_mismatching)

# soc_evolution_without_mismatching = equalize_series_length(soc_evolution_without_mismatching, soc_evolution_without_mismatching)
# soc_evolution_with_mismatching = equalize_series_length(soc_evolution_with_mismatching, soc_evolution_with_mismatching)

equ_net_grid_evolution_without_mismatching = equalize_series_length(net_grid_evolution_without_mismatching, soc_evolution_without_mismatching)

if isinstance(net_grid_evolution_with_mismatching[0], list): #perform only once
    equ_net_grid_evolution_with_mismatching = [sum(inner_list) for inner_list in net_grid_evolution_with_mismatching]
equ_net_grid_evolution_with_mismatching = equalize_series_length(equ_net_grid_evolution_with_mismatching, soc_evolution_with_mismatching)

equ_consumption = equalize_series_length(consumption_full, soc_evolution_without_mismatching)
equ_pv_production = equalize_series_length(pv_production_full, soc_evolution_without_mismatching)
equ_electricity_prices = equalize_series_length(electricity_prices_full, soc_evolution_without_mismatching)

#soc series could also be soc_evolution_with_mismatching

## Graph 1: consumption, production, prices, and net grid

In [ ]:
# Create a figure
fig = go.Figure()

# left y-axis
fig.add_trace(go.Scatter(x=list(range(len(equ_net_grid_evolution_without_mismatching))), y=equ_net_grid_evolution_without_mismatching, mode='lines', name='Grid Interaction without mismatching', line_shape='hv'))
fig.add_trace(go.Scatter(x=list(range(len(equ_net_grid_evolution_with_mismatching))), y=equ_net_grid_evolution_with_mismatching, mode='lines', name='Grid Interaction with mismatching (summed)', line_shape='hv'))

fig.add_trace(go.Scatter(x=list(range(len(equ_consumption))), y=equ_consumption, mode='lines', name='Consumption', line_shape='hv'))
fig.add_trace(go.Scatter(x=list(range(len(equ_pv_production))), y=equ_pv_production, mode='lines', name='PV Production', line_shape='hv'))
#right y-axis
fig.add_trace(go.Scatter(x=list(range(len(equ_electricity_prices))), y=equ_electricity_prices, mode='lines', name='Electricity Prices (right)', line_shape='hv', yaxis='y2'))

# Update the layout
fig.update_layout(title='Consumption, production, prices, and net grid',
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Value during timestep (kWh)'),
                yaxis2=dict(
                title='Price (Euro/kWh)',
                overlaying='y',
                side='right'),
                yaxis_title='Value during timestep (kWh)',
                xaxis=dict(dtick=1),
                legend_title='Legend',
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.2,  # Position legend at the top (y < 1)
                x=0.1)    # Center the legend horizontally )
                )

# Show the figure
fig.show()

Logic if prices high les grid interaction

Elec back to grid depends on consumption and production

## Graph 2: actions and soc

Logic it goes always to the extreme values

In [ ]:
fig = go.Figure()

# left y-axis
fig.add_trace(go.Scatter(x=list(range(len(equ_optimal_controls_without_mismatching))), y=equ_optimal_controls_without_mismatching, mode='lines', name='Battery actions without mismatching', line_shape='hv'))
#right y-axis
fig.add_trace(go.Scatter(x=list(range(len(soc_evolution_without_mismatching))), y=soc_evolution_without_mismatching, mode='lines', name='SoC without mismatching', yaxis='y2'))
fig.add_trace(go.Scatter(x=list(range(len(soc_evolution_with_mismatching))), y=soc_evolution_with_mismatching, mode='lines', name='SoC with mismatching (', yaxis='y2'))


# Create the x and y values for the continuous line
x_values_continuous = []
y_values_continuous = []

for index, data_list in enumerate(optimal_controls_with_mismatching):
    x_values_continuous.extend([index + i/(timesteps_for_mismatching -1) for i in range(len(data_list))])
    y_values_continuous.extend(data_list)
fig.add_trace(go.Scatter(x=x_values_continuous, y=y_values_continuous, mode='lines+markers', name='Battery actions with mismatching'))


# Update the layout
fig.update_layout(title='Actions and Soc battery',
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Action during timestep (kW)'),
                yaxis2=dict(
                title='Absolute battery level (kW), could be %',
                overlaying='y',
                side='right'),
                yaxis_title='Value during timestep (kWh)',
                xaxis=dict(dtick=1),
                legend_title='Legend',
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.2,  # Position legend at the top (y < 1)
                x=0.3)    # Center the legend horizontally )
                )

# Show the figure
fig.show()

Elbow is the timestep show a mismatch impact

## Graph 3: costs

Notes: computed before to be sure same length
- min heursitic is not really minimal because you call sell back to the grid. Minimal should take into account max price to sell back to the grid...
- grid sell back ratio, later maybe timeseries

In [ ]:
#heursitics
# rolling_max_heuristic, rolling_min_heuristic, rolling_avg_heuristic = rolling_problem.heuristic()
print(f"Max heuristic: {rolling_max_heuristic:.2f} Euro")
print(f"Min heuristic: {rolling_min_heuristic:.2f} Euro")
print(f"Average heuristic: {rolling_avg_heuristic:.2f} Euro")

In [ ]:
# I changed the cost functions... net grid is not correct anymore, too low values compared to 'difference'
total_cost_no_mismatching, buying_cost_no_mismatching, selling_cost_no_mismatching = cost_calculator_no_mismatching(net_grid_evolution_without_mismatching, electricity_prices_full, grid_price_ratio, N)
total_cost_with_mismatching, buying_cost_with_mismatching, selling_cost_with_mismatching = cost_calculator_with_mismatching(net_grid_evolution_with_mismatching, electricity_prices_full, grid_price_ratio, N)
print(f'total_cost_no_mismatching: {total_cost_no_mismatching}')
print(f'total_cost_with_mismatching: {total_cost_with_mismatching}')

In [ ]:
#make bar chart for total cost, buying cost, selling cost, max heuristic, min heuristic, avg heuristic using plotly
# Create bar chart
fig = go.Figure(data=[
    go.Bar(name='Total Cost no mismatching', x=['no_mismatch'], y=[total_cost_no_mismatching]),
    go.Bar(name='Buying Cost no mismatching', x=['no_mismatch'], y=[buying_cost_no_mismatching]),
    go.Bar(name='Selling savings no mismatching', x=['no_mismatch'], y=[selling_cost_no_mismatching]),

    go.Bar(name='Total Cost with mismatching', x=['mismatch'], y=[total_cost_with_mismatching]),
    go.Bar(name='Buying Cost with mismatching', x=['mismatch'], y=[buying_cost_with_mismatching]),
    go.Bar(name='Selling savings with mismatching', x=['mismatch'], y=[selling_cost_with_mismatching]),

    # go.Bar(name='random', x=['random'], y=[total_random_cost]),

    go.Bar(name='Max Heuristic', x=['heuristics'], y=[rolling_max_heuristic]),
    go.Bar(name='Min Heuristic', x=['heuristics'], y=[rolling_min_heuristic]),
    go.Bar(name='Avg Heuristic', x=['heuristics'], y=[rolling_avg_heuristic]),
])

# Update the layout
fig.update_layout(width=750, height=500,
    title='Costs and Heuristics Overview',
    xaxis_title='Category',
    yaxis_title='Value (Euro)',
    barmode='group',
    legend_title='Legend',
    bargap=0.05,  # gap between bars of adjacent location coordinates
)

# Show the figure
fig.show()
#make figure smaller


## Graph 4: optimisation process how convergence to solution

### 4.1 First timestep scheduling

In [ ]:
def split_into_sublists(flat_list, sublist_lengths):
    sublists = {}
    start = 0
    for i, length in enumerate(sublist_lengths, start=1):
        end = start + length
        sublists[f'guess_{i}'] = flat_list[start:end]
        start = end
    return sublists #dictionary

In [ ]:
original_sublists = split_into_sublists(no_loop_problem.evaluations, no_loop_list_nfev_guesses) #two ouput from optimization, note list_nfev_guesses could be also in class
# print(original_sublists['guess_20'])

In [ ]:
# Initialize the figure outside the loop
fig = go.Figure()

# Assuming original_sublists and N are defined
for i in range(1, num_guesses+1 ):
    fig.add_trace(go.Scatter(
        x=list(range(len(original_sublists[f'guess_{i}']))), 
        y=original_sublists[f'guess_{i}'], 
        mode='lines', 
        name=f'Guess {i}'
    ))

# Update layout once after all traces have been added
fig.update_layout(
    title='Visualization of evolution initial guesses',
    xaxis_title='Iterations',
    yaxis_title='Cost obtained by proposed actions (Euro)',
)

# Show the figure once after adding all traces
fig.show()


Note: MPC recomputing per timestep not a lot better (37.1) compared to scheduling everything at timestep 1. This is because no forecast variability for the moment.

### 4.2 Number of iterations per timestep

In [ ]:
nmber_iterations = [sum(inner_list) for inner_list in number_iterations_per_time_step]

colors = px.colors.qualitative.Set3  # Using Plotly's built-in qualitative color sets
# Ensure we have enough colors, repeat the list if necessary
colors *= len(nmber_iterations) // len(colors) + 1
# Create bar chart
fig = go.Figure(data=[
    go.Bar(name='Number of iterations', x=list(range(len(nmber_iterations))), y=nmber_iterations, marker_color=colors[:len(nmber_iterations)], width = 0.4),
])

fig.update_layout(
    title='Number of iterations per timestep',
    xaxis_title='Timestep',
    yaxis_title='Number of iterations',
    bargap=0.15  # Adjust the gap between bars, you can adjust as needed
)

fig.show()

## Graph 5: set-up overview

In [ ]:
# N = 24 #number of time steps in one day
# max_capacity = 10  # kWh
# max_charge_rate = 5  # kWh per hour (kW)
# max_discharge_rate = -5  
# efficiency_inverter = 1
# efficiency_battery = 1
# # Stored elec in battery and directly consumed(c) PV go through inverter
# # PV production (p) to battery no need to go through inverter


# # Sample data
# electricity_prices = np.random.uniform(0.1, 0.6, N)  # Euro per kWh
# pv_production = np.random.uniform(0, 5, N) # PV production (kWh)
# consumption = np.random.uniform(0, 20, N)  # Household consumption (kWh)

# # Initial state
# soc_start = 5  # not a prcenage, but a kWh value
# grid_price = 1 #later also time series

# #for guesses
# st_dev_fraction = 8
# num_guesses = 10 #later visualise how it influences the result
# # for MPC
# options = {'maxiter': 1000, 'ftol': 1e-6}
# method = 'SLSQP'


# #for loop or not and end condition (calculate on avg what is best to have, probably empty battery... because load by night so nothing to change)
# #later automation for mutiple days
# #forecast variability